1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
#Answer

#Binary to decimal
def btod(x):
    y = 0
    for i in range(0,len(x)):
        j = len(x)-(i+1)
        y = 2**i * int(x[j]) + y        
    return y 

#Decimal to binary
def dtob(x):
    res = 1
    y   = ''
    
    while res > 0:
        res = x // 2
        i   = x % 2
        x   = res
        y = str(i) + y
    return y

#Decimal to Hexadecimal
def dtoh(x):
    a = {0:'0', 1:'1', 2 :'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'A', 11:'B', 12:'C', 13:'D', 14:'E', 15:'F'}
    res = 1
    y   = ''
    
    while res > 0:
        res = x // 16
        i   = x % 16
        x   = res
        
        if i == 10:
            h=1
        y = a[i] + y
    return y
    


#Hexadecimal to decimal
def htod(x):
    y = 0
    a = {'0':0, '1':1, '2' :2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9,'A':10,'B':11,'C':12,'D':13,'E':14,'F':15}

    for i in range(0,len(x)):
        j = len(x)-(i+1)
        y = int(a[x[i]]) * 16**j +y
    return y

#Binary to hexadecimal
def btoh(x):
    y = dtoh(btod(x))
    return y

#Hexadecimal to binary
def htob(x):
    y = dtob(htod(x))
    return y

a = 29
print('Decimal number',a)
print('Decimal converted to binary,',dtob(a))
print('Decimal converted to hexadecimal,',dtoh(a))

print('--------------------------------------------')

b= '11101110'
print('Binary number',b)
print('Binary converted to decimal,',btod(b))
print('Binary converted to hexadecimal,',btoh(b))

print('--------------------------------------------')

c = '1DE3'
print('Hexadecimal number',c)
print('Hexadecimal converted to decimal,',htod(c))
print('Hexadecimal converted to binary,',htob(c))

Decimal number 29
Decimal converted to binary, 11101
Decimal converted to hexadecimal, 1D
--------------------------------------------
Binary number 11101110
Binary converted to decimal, 238
Binary converted to hexadecimal, EE
--------------------------------------------
Hexadecimal number 1DE3
Hexadecimal converted to decimal, 7651
Hexadecimal converted to binary, 1110111100011


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
#Answer
a = '00000011111000000000000000000000'
def bit32float(x):
    tmp = x [::-1]                      #Reverse the string to operate with conventional indexes 
    
    sgn = int(tmp[31])
    f = 1.
    for i in range(1,24):
        f = f + int(tmp[23-i]) * 2**(-i)
    e = 0.
    for i in range(0,8):
        e = e + int(tmp[23+i]) * 2**i
    
    y = (-1)**sgn * f * 2**(e-127)
    
    return y

print('The 32 bit word',a,'was converted into the floating point',bit32float(a))

The 32 bit word 00000011111000000000000000000000 was converted into the floating point 1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
#Answer

#Underflow
under = 1.
while under != 0.:
    underflow = under
    under /= 2
print('The underflow limit is:',underflow)

#Overflow
over = 1.
while over != float('inf'):
    overflow = over
    over *= 2
print('The overflow limit is :',overflow)

The underflow limit is: 5e-324
The overflow limit is : 8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
#Answer
epsilon = 1.

while epsilon +1. != 1.:
    epsilon /= 2
print('The machine precision is:',epsilon)

The machine precision is: 1.1102230246251565e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [5]:
#Answer
import  math as m
import  numpy as np

def bhaskara(a,b,c):
    if b**2-4*a*c < 0.:
        print('There is no real solution to the problem')
        return None 
    else: 
        x1 = (-b + m.sqrt(b**2-4*a*c))/(2*a)
        x2 = (-b - m.sqrt(b**2-4*a*c))/(2*a)
        return x1,x2

def bhaskara2(a,b,c):
    if b**2-4*a*c < 0.:
        print('There is no real solution to the problem')
        return None 
    else: 
        x1 = 2 *c /(-b-m.sqrt(b**2-4*a*c))
        x2 = 2 *c /(-b+m.sqrt(b**2-4*a*c))
        return x1,x2
    
def bhaskara3(a,b,c):
    if b**2-4*a*c < 0.:
        print('There is no real solution to the problem')
        return None 
    else: 
        x2 = (-b -np.sign(b)*m.sqrt(b**2 -4*a*c))/(2*a)
        x1 = c/(a*x2)
        return x1,x2
    
[a1,a2]=bhaskara(0.001,1000,0.001)
print('a) Solution using the Bhaskaras formula','         x1=',a1,'x2=',a2)

[a11,a22]=bhaskara2(0.001,1000,0.001)
print('b) Solution using the modified Bhaskaras formula','x1=',a11,'   x2=',a22)

print('')

print('How does it compare with what previously obtained? Why?')
print('Good question')

print('')
[a111,a222]=bhaskara3(0.001,1000,0.001)
print('c) Solution using the stable Bhaskaras formula','  x1=',a111,'        x2=',a222)

a) Solution using the Bhaskaras formula          x1= -9.999894245993346e-07 x2= -999999.999999
b) Solution using the modified Bhaskaras formula x1= -1.000000000001e-06    x2= -1000010.5755125057

How does it compare with what previously obtained? Why?
Good question

c) Solution using the stable Bhaskaras formula   x1= -1.000000000001e-06         x2= -999999.999999


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
#Answer
def f(x):
    y = x*(x-1.)
    return y

def derivate(f,x,d):
    df = (f(x+d)-f(x))/d
    return df
    
print('The analytical value of the derivative is:',1.)
print('The numerically obtained value of the derivative is:',derivate(f,1.,0.001), 'for delta =0.001')
print('a)  We can see a difference between the derivatives values, this happen because')
print('lolo')
print('------------------------------------------------')


delta = [10**-i for i in range(2,15,2)]
difference = [abs(1.-derivate(f,1.,delta[i])) for i in range(len(delta))]

print('b)  In this table it can be seen that the accuracy scales linearly with delta for delta between 1.e-2 and 1.e-8')
print('after this point this tendency is lost.')
print('Delta','|','Derivative','       |', 'Difference')
print('------------------------------------------------')
for i in range(len(delta)):
    print(delta[i],'|  ',derivate(f,1.,delta[i]),'|', difference[i])
    
    

The analytical value of the derivative is: 1.0
The numerically obtained value of the derivative is: 1.0009999999998895 for delta =0.001
a)  We can see a difference between the derivatives values, this happen because
lolo
------------------------------------------------
b)  In this table it can be seen that the accuracy scales linearly with delta for delta between 1.e-2 and 1.e-8
after this point this tendency is lost.
Delta | Derivative        | Difference
------------------------------------------------
0.01 |   1.010000000000001 | 0.010000000000000897
0.0001 |   1.0000999999998899 | 9.999999988985486e-05
1e-06 |   1.0000009999177333 | 9.99917733279787e-07
1e-08 |   1.0000000039225287 | 3.922528746258536e-09
1e-10 |   1.000000082840371 | 8.284037100736441e-08
1e-12 |   1.0000889005833413 | 8.890058334132256e-05
1e-14 |   0.9992007221626509 | 0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
#Answer
import  math as m
import  time

def scr1(x):
    y = m.sqrt(1-x**2)
    return y
    

def riemann(f,x1,x2,n):
    I = 0.
    for k in range(1,n):
        dx = (x2-x1)/n
        x  = x1 + dx*k
        I = I + dx * f(x)
    return I

print('The integral with N=100 is equal to',riemann(scr1,-1,1,100))
print('The difference with the true value is', abs(riemann(scr1,-1,1,100)-1.57079632679))


dt = 0.
rmn =0.
N=10
while dt < 1.:
    start =time.time()
    N *=10
    dprev = dt
    rmnprev =rmn
    rmn = riemann(scr1,-1,1,N)
    dt = abs(start - time.time())
print('If the computation must be done in less than a seccond, then N should be around',N, 'with a difference of',abs(rmnprev-1.57079632679))


dt = 0.
rmn =0.
N=10
while dt < 60.:
    start =time.time()
    N *=10
    dprev = dt
    rmnprev =rmn
    rmn = riemann(scr1,-1,1,N)
    dt = abs(start - time.time())
print('The gain is',abs(rmnprev-1.57079632679))


    
    

The integral with N=100 is equal to 1.5691342555492505
The difference with the true value is 0.0016620712407495741
If the computation must be done in less than a seccond, then N should be around 10000000 with a difference of 1.6582726303226991e-09


It means that to obtain a result 2 order more acurate takes 60 times more time what seems to be unnecessary considering that the accuracy obtained in 1 seccond is of nine digits.